In [16]:
# IMPORTS
import numpy as np
import pandas as pd
import os
import git
import sqlalchemy
import pyodbc 

In [17]:
# CONSTANTS
SOURCE_URL = 'https://github.com/CSSEGISandData/COVID-19'
DATA_PATH = '../data'
DAILY_REPORTS_PATH = f"{DATA_PATH}/csse_covid_19_data/csse_covid_19_daily_reports"
DAILY_REPORTS_TABLE_NAME = 'covid_daily_reports'
DAILY_REPORTS_US_TABLE_NAME = 'covid_daily_reports_us'
DAILY_REPORTS_US_PATH = f"{DATA_PATH}/csse_covid_19_data/csse_covid_19_daily_reports_us"
DAILY_REPORTS_COLUMN_NAMES = ['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incident_Rate', 'Case_Fatality_Ratio']
DAILY_REPORTS_US_COLUMN_NAMES = ['Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 
       'Confirmed', 'Deaths', 'Recovered', 'Active', 'FIPS', 'Incident_Rate', 'Total_Test_Results',
       'People_Hospitalized', 'Case_Fatality_Ratio', 'UID', 'ISO3', 'Testing_Rate', 'Hospitalization_Rate']


Настройки подключения к БД

In [ ]:
# DB CONFIG
SERVER_NAME = 'HOME-PC'
DB_NAME = 'covid'

Клонируем репозиторий с данными для этого нужно импортировать git.
Если гита нет то установить pip install gitpython

In [3]:
# Git clone
if not(os.path.exists(DATA_PATH)) or (not os.listdir(DATA_PATH)):
    git.Repo.clone_from(SOURCE_URL, DATA_PATH)

Настраиваем соединение с базой данны. Так как используем mssql нужен специальный драйвер для настройки соединения.
Также sqlalchemy поддерживает работу с pyodbc для более простой записи данных.

In [10]:
# DB connect
from sqlalchemy.engine import URL
connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER_NAME+';DATABASE='+DB_NAME+';Trusted_Connection=yes;'
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = sqlalchemy.create_engine(connection_url)

Далее описана функция для записи данных в таблицу.

In [54]:
def write_data(table_name, column_names, csv_folder_path, engine):
    for filename in os.listdir(csv_folder_path):
        if filename.endswith('.csv'):
            df = pd.read_csv(f"{csv_folder_path}/{filename}", header=None, skiprows=1)
            try:
                df.columns = column_names
                df.to_sql(
                    name=table_name,
                    con=engine,
                    index=False,
                    if_exists='append'
                )
            except:
                continue


Записываем данные в две разные таблицы

In [55]:
write_data(DAILY_REPORTS_TABLE_NAME, DAILY_REPORTS_COLUMN_NAMES, DAILY_REPORTS_PATH, engine)
write_data(DAILY_REPORTS_US_TABLE_NAME, DAILY_REPORTS_US_COLUMN_NAMES, DAILY_REPORTS_US_PATH, engine)